In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import datetime


from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.ensemble import HistGradientBoostingRegressor
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

# Initialize lists to collect model performance
model_names = []
model_maes = []
model_mabs = []

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

In [2]:
data_set = pd.read_csv('1.csv')
data_set.head()
data_set.tail()

,Date,SP500,Futures,Nikkei,FTSE,DAX
0,04/03/19,"2,873.40","2,879.75","21,713.21","7,418.28","11,954.40"
1,04/04/19,"2,879.39","2,882.75","21,724.95","7,401.94","11,988.01"
2,04/05/19,"2,892.74","2,896.00","21,807.50","7,446.87","12,009.75"
3,04/08/19,"2,895.77","2,898.25","21,761.65","7,451.89","11,963.40"
4,04/09/19,"2,878.20","2,882.50","21,802.59","7,425.57","11,850.57"


,Date,SP500,Futures,Nikkei,FTSE,DAX
1511,04/07/25,"5,062.25","5,149.75","31,136.58","7,702.08","19,789.62"
1512,04/08/25,"4,982.77","5,020.25","33,012.58","7,910.53","20,280.26"
1513,04/09/25,"5,456.90","5,491.00","31,714.03","7,679.48","19,670.88"
1514,04/10/25,"5,268.05","5,302.00","34,609.00","7,913.25","20,562.73"
1515,04/11/25,"5,363.36","5,391.25","33,585.58","7,964.18","20,374.10"


In [3]:
data_set.columns

# Step 1: Convert to datetime
data_set['Date'] = pd.to_datetime(data_set['Date'])

# ✅ Step 2: Extract day of week
data_set['day_of_week'] = data_set['Date'].dt.dayofweek  # 0=Mon, ..., 6=Sun

# Step 3 (optional): If you still want UNIX timestamp
data_set['Date'] = data_set['Date'].astype(np.int64) // 10**9

data_set.tail()

Index(['Date', 'SP500', 'Futures', 'Nikkei', 'FTSE', 'DAX'], dtype='object')

C:\Users\rajni\AppData\Local\Temp\ipykernel_21372\866144551.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_set['Date'] = pd.to_datetime(data_set['Date'])


,Date,SP500,Futures,Nikkei,FTSE,DAX,day_of_week
1511,1743984000,"5,062.25","5,149.75","31,136.58","7,702.08","19,789.62",0
1512,1744070400,"4,982.77","5,020.25","33,012.58","7,910.53","20,280.26",1
1513,1744156800,"5,456.90","5,491.00","31,714.03","7,679.48","19,670.88",2
1514,1744243200,"5,268.05","5,302.00","34,609.00","7,913.25","20,562.73",3
1515,1744329600,"5,363.36","5,391.25","33,585.58","7,964.18","20,374.10",4


In [4]:
# Reshape to 2D as required by OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
day_of_week_encoded = encoder.fit_transform(data_set[['day_of_week']])


In [5]:
# Get feature names like ['day_of_week_0', 'day_of_week_1', ...]
encoded_cols = encoder.get_feature_names_out(['day_of_week'])
encoded_df = pd.DataFrame(day_of_week_encoded, columns=encoded_cols, index=data_set.index)
data_set = pd.concat([data_set.drop(columns=['day_of_week']), encoded_df], axis=1)

In [6]:
def add_lag_features(df, columns, lags):
    """
    Adds lag features for given columns and lag days.

    Parameters:
    df (pd.DataFrame): Original dataset
    columns (list): Columns for which to create lag features
    lags (list): List of lag values (e.g., [1, 2, 3])

    Returns:
    pd.DataFrame: Dataset with new lag features
    """
    for col in columns:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    return df


lag_columns = ['SP500', 'DAX', 'FTSE', 'Nikkei']
lag_days = [1, 2, 3] 

data_set = add_lag_features(data_set, lag_columns, lag_days)
data_set = data_set.dropna()  # Drop rows with NaNs from lags
data_set.head()


data_set = data_set.sort_values(by='Date').reset_index(drop=True)



,Date,SP500,Futures,Nikkei,FTSE,DAX,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,...,SP500_lag_3,DAX_lag_1,DAX_lag_2,DAX_lag_3,FTSE_lag_1,FTSE_lag_2,FTSE_lag_3,Nikkei_lag_1,Nikkei_lag_2,Nikkei_lag_3
3,1554681600,"2,895.77","2,898.25","21,761.65","7,451.89","11,963.40",1.00,0.00,0.00,0.00,...,"2,873.40","12,009.75","11,988.01","11,954.40","7,446.87","7,401.94","7,418.28","21,807.50","21,724.95","21,713.21"
4,1554768000,"2,878.20","2,882.50","21,802.59","7,425.57","11,850.57",0.00,1.00,0.00,0.00,...,"2,879.39","11,963.40","12,009.75","11,988.01","7,451.89","7,446.87","7,401.94","21,761.65","21,807.50","21,724.95"
5,1554854400,"2,888.21","2,894.50","21,687.57","7,421.91","11,905.91",0.00,0.00,1.00,0.00,...,"2,892.74","11,850.57","11,963.40","12,009.75","7,425.57","7,451.89","7,446.87","21,802.59","21,761.65","21,807.50"
6,1554940800,"2,888.32","2,891.75","21,711.38","7,417.95","11,935.20",0.00,0.00,0.00,1.00,...,"2,895.77","11,905.91","11,850.57","11,963.40","7,421.91","7,425.57","7,451.89","21,687.57","21,802.59","21,761.65"
7,1555027200,"2,907.41","2,912.50","21,870.56","7,437.06","11,999.93",0.00,0.00,0.00,0.00,...,"2,878.20","11,935.20","11,905.91","11,850.57","7,417.95","7,421.91","7,425.57","21,711.38","21,687.57","21,802.59"


In [7]:
print(data_set.columns)

Index(['Date', 'SP500', 'Futures', 'Nikkei', 'FTSE', 'DAX', 'day_of_week_0',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'SP500_lag_1', 'SP500_lag_2', 'SP500_lag_3', 'DAX_lag_1', 'DAX_lag_2',
       'DAX_lag_3', 'FTSE_lag_1', 'FTSE_lag_2', 'FTSE_lag_3', 'Nikkei_lag_1',
       'Nikkei_lag_2', 'Nikkei_lag_3'],
      dtype='object')


In [8]:
# Create target columns first
data_set['SP500_next'] = data_set['SP500'].shift(-1)
data_set['Nikkei_next'] = data_set['Nikkei'].shift(-1)
data_set['FTSE_next'] = data_set['FTSE'].shift(-1)
data_set['DAX_next'] = data_set['DAX'].shift(-1)

# Now drop any rows with NaNs
data_set = data_set.dropna()


In [9]:
from sklearn.preprocessing import StandardScaler

# Drop the 'Date' column
data_scaled_input = data_set.drop(columns=['Date'])

# Apply StandardScaler
scaler = StandardScaler()
scaled_array = scaler.fit_transform(data_scaled_input)

# Convert the scaled data back to DataFrame
data_scaled = pd.DataFrame(scaled_array, columns=data_scaled_input.columns)

# Preview the result
data_scaled.head()

,SP500,Futures,Nikkei,FTSE,DAX,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,...,FTSE_lag_1,FTSE_lag_2,FTSE_lag_3,Nikkei_lag_1,Nikkei_lag_2,Nikkei_lag_3,SP500_next,Nikkei_next,FTSE_next,DAX_next
0,-1.46,-1.45,-1.26,0.17,-1.19,2.17,-0.49,-0.50,-0.49,-0.55,...,0.17,0.10,0.12,-1.25,-1.26,-1.26,-1.48,-1.25,0.14,-1.23
1,-1.48,-1.47,-1.25,0.14,-1.23,-0.46,2.03,-0.50,-0.49,-0.55,...,0.17,0.16,0.10,-1.26,-1.25,-1.26,-1.47,-1.27,0.13,-1.21
2,-1.47,-1.46,-1.27,0.13,-1.21,-0.46,-0.49,2.00,-0.49,-0.55,...,0.13,0.17,0.16,-1.25,-1.26,-1.25,-1.47,-1.27,0.12,-1.20
3,-1.47,-1.46,-1.26,0.13,-1.20,-0.46,-0.49,-0.50,2.02,-0.55,...,0.13,0.13,0.17,-1.27,-1.25,-1.26,-1.45,-1.24,0.15,-1.18
4,-1.45,-1.44,-1.24,0.15,-1.18,-0.46,-0.49,-0.50,-0.49,1.82,...,0.12,0.13,0.13,-1.26,-1.27,-1.25,-1.45,-1.19,0.15,-1.17


In [43]:
print(data_scaled.columns)

Index(['SP500', 'Futures', 'Nikkei', 'FTSE', 'DAX', 'day_of_week_0',
       'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'SP500_lag_1', 'SP500_lag_2', 'SP500_lag_3', 'DAX_lag_1', 'DAX_lag_2',
       'DAX_lag_3', 'FTSE_lag_1', 'FTSE_lag_2', 'FTSE_lag_3', 'Nikkei_lag_1',
       'Nikkei_lag_2', 'Nikkei_lag_3', 'SP500_next', 'Nikkei_next',
       'FTSE_next', 'DAX_next'],
      dtype='object')


In [10]:

# Parameters for sliding window
window_size = 10
train_size = 8
test_size = 2

X_full = data_scaled.drop(columns=['SP500_next'])
y_full = data_scaled['SP500_next']

# Containers to hold concatenated training and testing data
X_train_list, X_test_list = [], []
y_train_list, y_test_list = [], []

# Sliding window generation
for start in range(0, len(X_full) - window_size + 1, test_size):  # step by test size
    end = start + window_size
    X_window = X_full.iloc[start:end]
    y_window = y_full.iloc[start:end]
    
    X_train_list.append(X_window.iloc[:train_size])
    y_train_list.append(y_window.iloc[:train_size])
    X_test_list.append(X_window.iloc[train_size:])
    y_test_list.append(y_window.iloc[train_size:])

# Concatenate all windows together to get final train-test sets
X_train = pd.concat(X_train_list, ignore_index=True)
y_train = pd.concat(y_train_list, ignore_index=True)
X_test = pd.concat(X_test_list, ignore_index=True)
y_test = pd.concat(y_test_list, ignore_index=True)


In [11]:
# Parameters for sliding window
window_size = 10
train_size = 8
test_size = 2

X_full_nk = data_scaled.drop(columns=['Nikkei_next'])
y_full_nk = data_scaled['Nikkei_next']

# Containers to hold concatenated training and testing data
X_train_list_nk, X_test_list_nk = [], []
y_train_list_nk, y_test_list_nk = [], []

# Sliding window generation
for start in range(0, len(X_full_nk) - window_size + 1, test_size):  # step by test size
    end = start + window_size
    X_window_nk = X_full_nk.iloc[start:end]
    y_window_nk = y_full_nk.iloc[start:end]

    X_train_list_nk.append(X_window_nk.iloc[:train_size])
    y_train_list_nk.append(y_window_nk.iloc[:train_size])
    X_test_list_nk.append(X_window_nk.iloc[train_size:])
    y_test_list_nk.append(y_window_nk.iloc[train_size:])

# Concatenate all windows together to get final train-test sets
X_train_nk = pd.concat(X_train_list_nk, ignore_index=True)
y_train_nk = pd.concat(y_train_list_nk, ignore_index=True)
X_test_nk = pd.concat(X_test_list_nk, ignore_index=True)
y_test_nk = pd.concat(y_test_list_nk, ignore_index=True)


In [12]:
# Parameters for sliding window
window_size = 10
train_size = 8
test_size = 2

# Drop target from features and extract y values
X_full_fs = data_scaled.drop(columns=['FTSE_next'])
y_full_fs = data_scaled['FTSE_next']

# Lists to hold the training and testing splits
X_train_list_fs, X_test_list_fs = [], []
y_train_list_fs, y_test_list_fs = [], []

# Sliding window generation loop
for start in range(0, len(X_full_fs) - window_size + 1, test_size):  # step by test_size
    end = start + window_size
    X_window_fs = X_full_fs.iloc[start:end]
    y_window_fs = y_full_fs.iloc[start:end]

    X_train_list_fs.append(X_window_fs.iloc[:train_size])
    y_train_list_fs.append(y_window_fs.iloc[:train_size])
    X_test_list_fs.append(X_window_fs.iloc[train_size:])
    y_test_list_fs.append(y_window_fs.iloc[train_size:])

# Concatenate all windows together
X_train_fs = pd.concat(X_train_list_fs, ignore_index=True)
y_train_fs = pd.concat(y_train_list_fs, ignore_index=True)
X_test_fs = pd.concat(X_test_list_fs, ignore_index=True)
y_test_fs = pd.concat(y_test_list_fs, ignore_index=True)


In [13]:
# Parameters for sliding window
window_size = 10
train_size = 8
test_size = 2

# Drop target from features and extract y values
X_full_dx = data_scaled.drop(columns=['DAX_next'])
y_full_dx = data_scaled['DAX_next']

# Lists to hold the training and testing splits
X_train_list_dx, X_test_list_dx = [], []
y_train_list_dx, y_test_list_dx = [], []

# Sliding window generation loop
for start in range(0, len(X_full_dx) - window_size + 1, test_size):  # step by test_size
    end = start + window_size
    X_window_dx = X_full_dx.iloc[start:end]
    y_window_dx = y_full_dx.iloc[start:end]

    X_train_list_dx.append(X_window_dx.iloc[:train_size])
    y_train_list_dx.append(y_window_dx.iloc[:train_size])
    X_test_list_dx.append(X_window_dx.iloc[train_size:])
    y_test_list_dx.append(y_window_dx.iloc[train_size:])

# Concatenate all windows together
X_train_dx = pd.concat(X_train_list_dx, ignore_index=True)
y_train_dx = pd.concat(y_train_list_dx, ignore_index=True)
X_test_dx = pd.concat(X_test_list_dx, ignore_index=True)
y_test_dx = pd.concat(y_test_list_dx, ignore_index=True)


# Hybrid for S&P500

In [14]:
model = LinearRegression(fit_intercept = True)
model.fit(X_train, y_train) 

# The following gives the R-square score
model.score(X_train, y_train) 

LinearRegression()

0.9977129875863636

In [15]:
training_residuals = y_train - model.predict(X_train)

In [16]:
rf = RandomForestRegressor(random_state=50, min_samples_leaf = 3, max_features = "sqrt")

rf = rf.fit(X_train, training_residuals) 

In [17]:
pred_residuals = rf.predict(X_test)
y_pred = pred_residuals + model.predict(X_test)

In [18]:
test_output = pd.DataFrame(y_pred, index = X_test.index, columns = ['pred_SP500_next'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_SP500_next'] - test_output['SP500_next']).mean() / test_output['SP500_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Hybrid Model")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

,pred_SP500_next,SP500_next
0,-1.42,-1.41
1,-1.48,-1.48
2,-1.55,-1.56
3,-1.53,-1.53
4,-1.52,-1.52


Mean absolute error is 
0.0048956114905216305
Mean absolute percentage error is 
0.45609317464218363


In [19]:
from sklearn.metrics import r2_score

# Training R²
train_hybrid_pred = model.predict(X_train) + rf.predict(X_train)
hybrid_r2_train = r2_score(y_train, train_hybrid_pred)
print(f"Hybrid Model R² score on training set: {hybrid_r2_train:.4f}")

# Test R² (y_pred is already calculated above)
hybrid_r2_test = r2_score(y_test, y_pred)
print(f"Hybrid Model R² score on test set: {hybrid_r2_test:.4f}")


Hybrid Model R² score on training set: 0.9999
Hybrid Model R² score on test set: 0.9996


# Hybrid Model for Nikkei

In [20]:
model_nk = LinearRegression(fit_intercept = True)
model_nk.fit(X_train_nk, y_train_nk) 

# The following gives the R-square score
model_nk.score(X_train_nk, y_train_nk) 

LinearRegression()

0.9958868135766856

In [21]:
training_residuals_nk = y_train_nk - model_nk.predict(X_train_nk)

In [22]:
rf_nk = RandomForestRegressor(random_state=50, min_samples_leaf = 3, max_features = "sqrt")

rf_nk = rf_nk.fit(X_train_nk, training_residuals_nk) 

In [23]:
pred_residuals_nk = rf_nk.predict(X_test_nk)
y_pred_nk = pred_residuals_nk + model_nk.predict(X_test_nk)

In [24]:
test_output = pd.DataFrame(y_pred_nk, index = X_test_nk.index, columns = ['pred_Nikkei_next'])
test_output = test_output.merge(y_test_nk, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_Nikkei_next'] - test_output['Nikkei_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_Nikkei_next'] - test_output['Nikkei_next']).mean() / test_output['Nikkei_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Hybrid Model")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

,pred_Nikkei_next,Nikkei_next
0,-1.18,-1.17
1,-1.30,-1.33
2,-1.35,-1.36
3,-1.38,-1.38
4,-1.36,-1.36


Mean absolute error is 
0.006176728467144619
Mean absolute percentage error is 
0.689044196642292


In [25]:
from sklearn.metrics import r2_score

# Predict hybrid values on training set
train_hybrid_pred_nk = model_nk.predict(X_train_nk) + rf_nk.predict(X_train_nk)
hybrid_r2_train_nk = r2_score(y_train_nk, train_hybrid_pred_nk)
print(f"Hybrid Model R² score on training set (Nikkei): {hybrid_r2_train_nk:.4f}")

# Predict hybrid values on test set (y_pred_nk is already calculated)
hybrid_r2_test_nk = r2_score(y_test_nk, y_pred_nk)
print(f"Hybrid Model R² score on test set (Nikkei): {hybrid_r2_test_nk:.4f}")


Hybrid Model R² score on training set (Nikkei): 0.9999
Hybrid Model R² score on test set (Nikkei): 0.9997


# Hybrid for FTSE 

In [26]:
model_fs = LinearRegression(fit_intercept = True)
model_fs.fit(X_train_fs, y_train_fs) 

# The following gives the R-square score
model_fs.score(X_train_fs, y_train_fs) 

LinearRegression()

0.9949807345866856

In [27]:
training_residuals_fs = y_train_fs - model_fs.predict(X_train_fs)

In [28]:
rf_fs = RandomForestRegressor(random_state=50, min_samples_leaf = 3, max_features = "sqrt")

rf_fs = rf_fs.fit(X_train_fs, training_residuals_fs) 

In [29]:
pred_residuals_fs = rf_fs.predict(X_test_fs)
y_pred_fs = pred_residuals_fs + model_fs.predict(X_test_fs)

In [30]:
test_output = pd.DataFrame(y_pred_fs, index = X_test_fs.index, columns = ['pred_FTSE_next'])
test_output = test_output.merge(y_test_fs, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_FTSE_next'] - test_output['FTSE_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_FTSE_next'] - test_output['FTSE_next']).mean() / test_output['FTSE_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Gradient Boosting Regressor")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

,pred_FTSE_next,FTSE_next
0,0.15,0.14
1,-0.16,-0.19
2,-0.25,-0.24
3,-0.14,-0.13
4,-0.05,-0.05


Mean absolute error is 
0.006644434121253117
Mean absolute percentage error is 
-5.651069329802366


In [31]:
from sklearn.metrics import r2_score

# Predict hybrid values on training set
train_hybrid_pred_fs = model_fs.predict(X_train_fs) + rf_fs.predict(X_train_fs)
hybrid_r2_train_fs = r2_score(y_train_fs, train_hybrid_pred_fs)
print(f"Hybrid Model R² score on training set (FTSE): {hybrid_r2_train_fs:.4f}")

# Predict hybrid values on test set
hybrid_r2_test_fs = r2_score(y_test_fs, y_pred_fs)
print(f"Hybrid Model R² score on test set (FTSE): {hybrid_r2_test_fs:.4f}")



Hybrid Model R² score on training set (FTSE): 0.9999
Hybrid Model R² score on test set (FTSE): 0.9998


# Hybrid for DAX

In [32]:
model_dx = LinearRegression(fit_intercept = True)
model_dx.fit(X_train_dx, y_train_dx) 

# The following gives the R-square score
model_dx.score(X_train_dx, y_train_dx) 

LinearRegression()

0.9980306172287206

In [33]:
training_residuals_dx = y_train_dx - model_dx.predict(X_train_dx)

In [34]:
rf_dx = RandomForestRegressor(random_state=50, min_samples_leaf = 3, max_features = "sqrt")

rf_dx = rf_dx.fit(X_train_dx, training_residuals_dx) 

In [35]:
pred_residuals_dx = rf_dx.predict(X_test_dx)
y_pred_dx = pred_residuals_dx + model_dx.predict(X_test_dx)

In [36]:
test_output = pd.DataFrame(y_pred_dx, index = X_test_dx.index, columns = ['pred_DAX_next'])
test_output = test_output.merge(y_test_dx, left_index = True, right_index = True)
test_output.head()
mean_absolute_error = abs(test_output['pred_DAX_next'] - test_output['DAX_next']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
mean_absolute_percentage_error = abs(test_output['pred_DAX_next'] - test_output['DAX_next']).mean() / test_output['DAX_next'].mean()
print('Mean absolute percentage error is ')
print(mean_absolute_percentage_error)

model_names.append("Hybrid Model DAX")
model_maes.append(mean_absolute_error)
model_mabs.append(mean_absolute_percentage_error)

,pred_DAX_next,DAX_next
0,-1.06,-1.06
1,-1.16,-1.15
2,-1.22,-1.22
3,-1.18,-1.18
4,-1.14,-1.14


Mean absolute error is 
0.004154191720627976
Mean absolute percentage error is 
0.4843725032052141


In [37]:
from sklearn.metrics import r2_score

# Predict hybrid values on training set
train_hybrid_pred_dx = model_dx.predict(X_train_dx) + rf_dx.predict(X_train_dx)
hybrid_r2_train_dx = r2_score(y_train_dx, train_hybrid_pred_dx)
print(f"Hybrid Model R² score on training set (DAX): {hybrid_r2_train_dx:.4f}")

# Predict hybrid values on test set
hybrid_r2_test_dx = r2_score(y_test_dx, y_pred_dx)
print(f"Hybrid Model R² score on test set (DAX): {hybrid_r2_test_dx:.4f}")


Hybrid Model R² score on training set (DAX): 0.9999
Hybrid Model R² score on test set (DAX): 0.9999


In [44]:
import datetime
import pandas as pd

# Step 1: Get full feature list model was trained on
required_features = model.feature_names_in_

# Step 2: Get last row
last_known = data_set.iloc[-1].copy()

# Step 3: Generate day_of_week features
future_date = pd.to_datetime(datetime.date.today()) + pd.Timedelta(days=1)
dow = future_date.weekday()
dow_features = {f'day_of_week_{i}': int(i == dow) for i in range(5)}

# Step 4: Prepare dictionary for all 25 features expected by the model
input_dict = {}
for col in required_features:
    if col in dow_features:
        input_dict[col] = dow_features[col]
    else:
        input_dict[col] = last_known[col]

# Step 5: Predict
input_df = pd.DataFrame([input_dict])[required_features]
base_pred = model.predict(input_df)[0]
residual_pred = rf.predict(input_df)[0]
hybrid_pred = base_pred + residual_pred

# Step 6: Output
print(f"Predicted S&P500 for {future_date.date()} = {hybrid_pred:.2f}")


Predicted S&P500 for 2025-04-23 = 5591.74
